In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from auction.operations import create_auction_app, setup_auction_app, place_bid, close_auction
from utils import get_algod_client, wait_for_confirmation, get_balances, get_last_block_timestamp
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
def optin_to_asset(asset_id, sender: Account):
    txn = transaction.AssetOptInTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=asset_id
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    wait_for_confirmation(client, signed_txn.get_txid())

In [4]:
creator = Account(os.environ.get('CREATOR_PK'))
seller = Account(os.environ.get('SELLER_PK'))

print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: FE6UTVPOXD7HCTEYTG27P7KEDZCQFX7ECJANPTI76CMJSKKDQRYFVB4NWM
Seller Address: GYZIWMWOUMBIUUI3QV3VRWQYVJ5QGMO7CUW4RNMCIZKGJULFOFQFF3MCGA


Alice is generating an example token...

In [5]:
def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
    txn = transaction.AssetConfigTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        total=total,
        decimals=decimals,
        asset_name=asset_name,
        unit_name=unit_name,
        default_frozen=False,
        strict_empty_address_check=False,
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    response = wait_for_confirmation(client, signed_txn.get_txid())
    assert response.asset_index is not None and response.asset_index > 0
    return response.asset_index

asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")
print(f"The token id is: {asset_id}")

Waiting for confirmation...
Transaction XU6KLFOA7ABWDBJHXFC7EMUEJVEZD25WRU65NNPMG4ZVTMI4EJZA confirmed in round 256.
The token id is: 1


In [6]:
start_time = int(time()) + 10
end_time = start_time + 10
reserve = 1_000_000  # 1 Algo
increment = 100_000  # 0.1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [7]:
appID = create_auction_app(
    client=client,
    sender=creator,
    seller=seller.get_address(),
    token_id=asset_id,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment,
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction 6IS3DIXQRS5EBVZHYQ2BO5AJFDXBLVWHX5R6RKAI3UYZCOVEX6TA confirmed in round 258.
App ID: 2
App Address: FHQPLJVRO7FVHYKCA2SGR3I7ZHHKHAQS4AGOAVGGGJQJYTTBVEJUGHN5JQ


Alice is setting up and funding token auction...

In [8]:
setup_auction_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_holder=seller,
    token_id=asset_id,
    asset_amount=1
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction OWBWZEJXVUV55Q4HE7W5SSNF6ID76X3DGQ35XIZERKCNRK467UWQ confirmed in round 260.
Alice's algo balance:  4000023999998000  algos


In [9]:
bidder = Account(os.environ.get("BUYER_PK"))

print(f"Bidder Address: {bidder.get_address()}")

Bidder Address: PAGKBMWTKVNLVPOT6JQLMMCEULFMOMGNU3JU6GFISDKTFMRAYOJPBARU5Y


In [10]:
_, last_round_time = get_last_block_timestamp(client)
if last_round_time < start_time + 5:
    sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 202000, 1: 1}


In [11]:
bidAmount = reserve
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

Carla wants to bid on token, her algo balance:  1000006000000000  algos
Carla is placing bid for:  1000000  algos


In [12]:
place_bid(client=client, app_id=appID, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

Waiting for confirmation...
Transaction 7T3C3LYT7EJZHRBNHEV3A4FHPS6M4MSS3HQTBV2HGTT5F2XTQLIQ confirmed in round 263.
Carla is opting into token with id: 1


In [13]:
optin_to_asset(asset_id, bidder)

Waiting for confirmation...
Transaction S23Z2WMPE6Y3SCZV4WEFKE2Y5AM3S4KY4CEO4OPTBVYZHVYKUZVA confirmed in round 265.


Alice is closing out the auction....

In [14]:
_, lastRoundTime = get_last_block_timestamp(client)
if lastRoundTime < end_time + 5:
    sleep(end_time + 5 - lastRoundTime)

close_auction(client, appID, seller)

Waiting for confirmation...
Transaction LIV5KSITX4ORH424JXPKLX4MXDI4JDAEQWD23W6CYGBBRSYRB54A confirmed in round 267.


In [15]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 0}


In [16]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 1  for token ID:  1
Alice's balances after auction:  {0: 4000024001197000, 1: 0}  Algos
Carla's balances after auction:  {0: 1000005998997000, 1: 1}  Algos
